# 0. Imports and Arguments


In [260]:
import os
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns;

from scripts.evaluation_utils import delayed_impact_csv, immediate_impact_csv, delayed_impact_german_csv,types_csvs

In [321]:
data_path = '../final_results/fico_data/syn_orig/ii/'
folders= ['dt','gnb','lgr','gbt']

# 1. Build useful CSVs

## 1.2 FP/TP/TN/FN Ratios

In [322]:
types_csvs(data_path, folders)

## 1.1 Impact

In [323]:
delayed_impact_csv(data_path,0, folders)
delayed_impact_csv(data_path,1, folders)

Group:  0 
 DataFrame: 
                DT   GNB   LGR   GBT
Constraint                         
Unmitigated  0.10  3.50 -7.11  0.02
DP           0.62 -4.57 -7.40 -0.13
EO          -0.44 -1.97 -5.52  0.35
EOO          0.35 -8.30 -6.91 -0.21
FPER        -1.10 -5.21 -6.45 -0.11
ERP         -0.27  2.85 -6.31  0.08
Group:  1 
 DataFrame: 
                 DT    GNB    LGR    GBT
Constraint                             
Unmitigated  37.04  24.87  36.16  37.18
DP           37.37  18.07  38.57  37.50
EO           37.03  34.30  33.96  36.99
EOO          38.61  36.38  36.66  38.68
FPER         37.62  36.08  36.28  37.47
ERP          33.57  27.17  31.29  33.92


In [264]:
immediate_impact_csv(data_path,0, folders)
immediate_impact_csv(data_path,1, folders)

Group:  0 
 DataFrame: 
                DT    GNB    LGR   GBT
Constraint                           
Unmitigated  0.49  0.472  0.529  0.49
DP           0.49  0.503  0.536  0.49
EO           0.49  0.505  0.526  0.49
EOO          0.49  0.530  0.532  0.49
FPER         0.49  0.525  0.529  0.49
ERP          0.49  0.472  0.529  0.49
Group:  1 
 DataFrame: 
                 DT    GNB    LGR    GBT
Constraint                             
Unmitigated  0.647  0.743  0.672  0.643
DP           0.508  0.311  0.585  0.505
EO           0.569  0.583  0.599  0.565
EOO          0.573  0.607  0.615  0.571
FPER         0.635  0.647  0.665  0.631
ERP          0.573  0.737  0.599  0.585


# 2. Analyzing Scores (only fico_data)

## 2.1 Extracting Scores from csv into dataframes

In [266]:
# Scores Data Frames
classifier_dfs = {}
dfs_b = {}
dfs_w = {}
# loading test set credit scores into dictinary from all models
for f in folders:
    path = f'{data_path}{f}/{f}_all_scores.csv'
    df = pd.read_csv(path)
    df = df.reset_index(drop=True)
    df = df.round(0)

    df_black = df.filter(like='B')
    df_white = df.filter(like='W')
    
    classifier_dfs[f] = df
    dfs_b[f] = df_black
    dfs_w[f] = df_white

## 2.2 Checking for normal distributions:

if p < 0.01 (or < 0.05) then the distribution is significantly different from a normal distribution

In [267]:
for c,df in classifier_dfs.items():
    print('Classifier:',c)
    for col in df:
        data=df[col].dropna(axis=0)
        _,p = stats.kstest(data, "norm") # comparing score distribution to normal distribution
        if p > 0.01:
            print(col,',p:',p)
    print('Check for normal distributions->done')

Classifier: dt
Check for norm Distributions done
Classifier: gnb
Check for norm Distributions done
Classifier: lgr
Check for norm Distributions done
Classifier: gbt
Check for norm Distributions done


## 2.3 Significance of Score Distributions with Mann Whitney U test:


In [268]:
mwu_path = f'{data_path}mwu/'
os.makedirs(mwu_path,exist_ok = True)

### Variance of Distributions unmitigated v mitigated for each race

if p < 0.001 (or < 0.0005) then the distributions are significantly different from each other

In [269]:
def p_race_mwu(dfs, b_or_w = 'B'):
    p_vals = pd.DataFrame(data={'Constraints': []})
    p_signi = pd.DataFrame(data={'Constraints': []})
    
    for c,df in dfs.items():
        
        c = f'{c}{b_or_w}'
        col_signi = []
        col_vals = []
        idx = []
        
        data_unmiti = df[f'unmit{b_or_w}'].dropna(axis=0) # extract scores from unmitigated models
        df = df.iloc[:,1:]
        for col in df:
            
            idx.append(col[:-1])
            
            data_miti=df[col].dropna(axis=0) # extract scores from mitigated models
            
            _,p = stats.mannwhitneyu(data_unmiti, data_miti) # compare unmitigated and mitigated models
            col_vals.append(p)
            
            # p values < 0.05 the difference is significat
            if p< 0.05:
                col_signi.append('s')
            else:
                col_signi.append(' ')
                
        p_signi[c] = col_signi
        p_vals[c] = col_vals
    # set index
    p_vals['Constraints'] = idx
    p_vals = p_vals.set_index('Constraints')
    
    p_signi['Constraints'] = idx
    p_signi = p_signi.set_index('Constraints')
    
    p_vals = p_vals.round(decimals=3)
    print(p_signi)
    # save p_values and significance for all models
    p_vals.to_csv(f'{mwu_path}p_un_vs_miti_{b_or_w}.csv')
    p_signi.to_csv(f'{mwu_path}significanz_un_vs_miti_{b_or_w}.csv')
   

In [270]:
print('Black:')
p_race_mwu(dfs_b,'B')

print('\nWhite:')
p_race_mwu(dfs_w,'W')

Black:
            dtB gnbB lgrB gbtB
Constraints                   
unmit                         
dp                 s          
eo                 s          
tprp               s          
fprp               s          
erp                           

White:
            dtW gnbW lgrW gbtW
Constraints                   
unmit                         
dp                 s          
eo                            
tprp                          
fprp                          
erp                           
